In [1]:
from collections import Counter
from datetime import datetime
from dateutil.relativedelta import relativedelta
import logging
import re
import numpy as np
import pandas as pd
import os
import pickle

In [35]:
from scipy import stats

In [46]:
def get_slop(df):
    x = [x + 1for x in range(0,len(list(df)))]
    slopes = []
    for i in df.values:
        y = i
        slope, intercept, r, p, std_err = stats.linregress(x, y)
        slopes.append(slope)
    return slopes

In [47]:
def data_pro(data,from_date,to_date):
    df = data.explode('keyword').reset_index(drop=True)
    df = df.groupby(['date','keyword']).size().reset_index()
    df = pd.pivot_table(df, index = ['keyword'], columns = 'date')
    df = df.fillna(0)
    df.columns = [x[1] for x in df.columns]
    return df

In [48]:
def DataKr(topic,from_date,to_date):
    cwd = os.getcwd()
    path = cwd +'/' + topic + '/raw'
    file_list = os.listdir(path)
    
    dfs = []
    for i in file_list:
        df = pd.read_excel(path+'/'+i)
        dfs.append(df)
        print(i)
    data = pd.concat(dfs)
    print("총 기사 건 수 : " + str(len(data)))  
    
    data = data[['뉴스 식별자','일자','제목','키워드','특성추출(가중치순 상위 50개)']]
    cluster_mode = '특성추출(가중치순 상위 50개)'
    corpus = data[cluster_mode].values.tolist()

    data_k = data[['일자']]

    for i, cp in enumerate(corpus):
        corpus[i] = cp.split(',')
    data_k['키워드'] = corpus
    
    df = data_k[['일자','키워드']]
    df = df.rename({'키워드':'keyword','일자':'date'},axis = 1 )
    df['date'] = [str(x)[0:6] for x in df['date']] 
    
    df_k = data_pro(df,from_date,to_date)
    df_k.to_csv(cwd +'/' + topic + '/'+ 'Monthly_Count_Keyword {}-{}.csv'.format(from_date,to_date),encoding="utf-8-sig")

    slopes = get_slop(df_k)
    df_ks = df_k.copy()
    df_ks['slope'] = slopes
    df_ks.to_csv(cwd +'/' + topic + '/'+ 'Efficient_OLS_Keyword {}-{}.csv'.format(from_date,to_date),encoding="utf-8-sig")

In [49]:
from_date = '20020101'
to_date = '20230430'
searching_word = '(농업 OR 농촌) AND 기후위기'

DataKr(searching_word,from_date ,to_date)

C:\Users\NH\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20020101-20230430.xlsx
총 기사 건 수 : 2677


C:\Users\NH\AppData\Local\Temp\ipykernel_23868\1633714248.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_k['키워드'] = corpus


In [50]:
from_date = '20020101'
to_date = '20230430'
searching_word = '(농업 OR 농촌) AND 기후변화'

DataKr(searching_word,from_date ,to_date)

NewsResult_20020101-20211231.xlsx
NewsResult_20220101-20230430.xlsx
총 기사 건 수 : 20904


C:\Users\NH\AppData\Local\Temp\ipykernel_23868\1633714248.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_k['키워드'] = corpus
